In [2]:
import pandas as pd
import numpy as np
path = 'D:/berkeley/290/ppp/'
sku = pd.read_csv(path + 'jdata_sku_basic_info.csv')
action = pd.read_csv(path + 'jdata_user_action.csv')
user = pd.read_csv(path + 'jdata_user_basic_info.csv')
comment = pd.read_csv(path + 'jdata_user_comment_score.csv')
order = pd.read_csv(path + 'jdata_user_order.csv')

In [1]:
def mean_notzero(series):
    series = [int(item) for item in series if int(item)!=0]
    if len(series) == 0:
        return 0
    else:
        return np.mean(series)

In [3]:
order['o_date']=pd.to_datetime(order['o_date'])
action['a_date']=pd.to_datetime(action['a_date'])
comment['c_date']=pd.to_datetime(comment['comment_create_tm']) 

time_start = pd.to_datetime('2016-05-01')
time_end = pd.to_datetime('2017-05-01')
time_period = pd.date_range(time_start,time_end,freq = '1D')

In [5]:
sku_list = list(sku['sku_id'])
user_new = user[['user_id','user_lv_cd']]
user_order = pd.merge(order,user_new,on='user_id')

In [7]:
for i in sku_list[30000:50000]:
    df = pd.DataFrame(columns = ['sku_id','order_time','sales'])
    df['order_time'] = time_period
    df['sku_id'] = i
    df.fillna(0,inplace = True)

    ##sales
    record = order[order['sku_id'] == i]
    if len(record) < 20:
        continue
    record = record[['o_date','o_sku_num']].groupby('o_date').sum()
    date_list = list(record.index)
    for date in date_list:
        df.loc[df['order_time'] == date,'sales'] = int(record.loc[date])

    df['sales_sum/2w'] = df['sales'].rolling(14,min_periods = 1).sum()
    df['sales_sum/month'] = df['sales'].rolling(30,min_periods = 1).sum()
    df['sales_sum/quarter'] = df['sales'].rolling(90,min_periods = 1).sum()
    df['sales_sum/half-year'] = df['sales'].rolling(180,min_periods = 1).sum()

    df['sales_mean/2w'] = df['sales'].rolling(14,min_periods = 1).apply(mean_notzero)
    df['sales_mean/month'] = df['sales'].rolling(30,min_periods = 1).apply(mean_notzero)
    df['sales_mean/quarter'] = df['sales'].rolling(90,min_periods = 1).apply(mean_notzero)
    df['sales_mean/half-year'] = df['sales'].rolling(180,min_periods = 1).apply(mean_notzero)
    
    ###comment
    comment['comment_time'] = pd.to_datetime(comment['comment_create_tm']).dt.date
    order_id = order.loc[order['sku_id'] == i,:]['o_id']
    order_id = list(order_id)
    comment_record = comment[comment['o_id'].isin(order_id)]

    comment_record = comment_record[comment_record['score_level'] != -1]
    comment_record_mean = comment_record[['comment_time','score_level']].groupby('comment_time').mean()
    comment_record_high = comment_record[['comment_time','score_level']].groupby('comment_time').min()
    comment_record_low = comment_record[['comment_time','score_level']].groupby('comment_time').max()

    df['comment_mean'] = 0
    df['comment_high'] = 0
    df['comment_low'] = 0

    date_list = list(comment_record_mean.index)
    for date in date_list:
        df.loc[df['order_time'] == date,'comment_mean'] = int(comment_record_mean.loc[date])
        df.loc[df['order_time'] == date,'comment_high'] = int(comment_record_high.loc[date])
        df.loc[df['order_time'] == date,'comment_low'] = int(comment_record_low.loc[date])

    df['comment_mean/2w'] = df['comment_mean'].rolling(14,min_periods = 1).apply(mean_notzero)
    df['comment_mean/month'] = df['comment_mean'].rolling(30,min_periods = 1).apply(mean_notzero)
    df['comment_mean/quarter'] = df['comment_mean'].rolling(90,min_periods = 1).apply(mean_notzero)
    df['comment_mean/half-year'] = df['comment_mean'].rolling(180,min_periods = 1).apply(mean_notzero)

    df['comment_high/2w'] = df['comment_high'].rolling(14,min_periods = 1).min()
    df['comment_high/month'] = df['comment_high'].rolling(30,min_periods = 1).min()
    df['comment_high/quarter'] = df['comment_high'].rolling(90,min_periods = 1).min()
    df['comment_high/half-year'] = df['comment_high'].rolling(180,min_periods = 1).min()

    df['comment_low/2w'] = df['comment_low'].rolling(14,min_periods = 1).max()
    df['comment_low/month'] = df['comment_low'].rolling(30,min_periods = 1).max()
    df['comment_low/quarter'] = df['comment_low'].rolling(90,min_periods = 1).max()
    df['comment_low/half-year'] = df['comment_low'].rolling(180,min_periods = 1).max()
    
    
    ##action
    browse_record = action[(action['sku_id'] == i)&(action['a_type'] == 1)]
    follow_record = action[(action['sku_id'] == i)&(action['a_type'] == 2)]

    browse_record = browse_record[['a_date','a_num']].groupby('a_date').sum()
    follow_record = follow_record[['a_date','a_num']].groupby('a_date').sum()

    df['browse'] = 0
    df['follow'] = 0
    browse_date_list = list(browse_record.index)
    for date in browse_date_list:
        df.loc[df['order_time'] == date,'browse'] = int(browse_record.loc[date])
    df['browse/2w'] = df['browse'].rolling(14,min_periods = 1).sum()
    df['browse/month'] = df['browse'].rolling(30,min_periods = 1).sum()
    df['browse/quarter'] = df['browse'].rolling(90,min_periods = 1).sum()
    df['browse/half-year'] = df['browse'].rolling(180,min_periods = 1).sum()

    follow_date_list = list(follow_record.index)
    for date in follow_date_list:
        df.loc[df['order_time'] == date,'follow'] = int(follow_record.loc[date])
    df['follow/2w'] = df['follow'].rolling(14,min_periods = 1).sum()
    df['follow/month'] = df['follow'].rolling(30,min_periods = 1).sum()
    df['follow/quarter'] = df['follow'].rolling(90,min_periods = 1).sum()
    df['follow/half-year'] = df['follow'].rolling(180,min_periods = 1).sum()
    
    
    ##user level
    user_record = user_order[user_order['sku_id'] == i]
    user_record = user_record[['o_date','user_lv_cd']].groupby('o_date').mean()

    df['user'] = 0
    user_date_list = list(user_record.index)
    for date in user_date_list:
        df.loc[df['order_time'] == date,'user'] = int(user_record.loc[date])
    df['user/2w'] = df['user'].rolling(14,min_periods = 1).apply(mean_notzero)
    df['user/month'] = df['user'].rolling(30,min_periods = 1).apply(mean_notzero)
    df['user/quarter'] = df['user'].rolling(90,min_periods = 1).apply(mean_notzero)
    df['user/half-year'] = df['user'].rolling(180,min_periods = 1).apply(mean_notzero)

    df['user_mean/2w'] = df['user'].rolling(14,min_periods = 1).mean()
    df['user_mean/month'] = df['user'].rolling(30,min_periods = 1).mean()
    
    ##location
    location = order[order['sku_id'] == i]
    location_record = location.groupby('o_date').o_area.nunique()

    df['area'] = 0
    location_date_list = list(location_record.index)
    for date in location_date_list:
        df.loc[df['order_time'] == date,'area'] = int(location_record[date])
    df['area_mean/2w'] = df['area'].rolling(14,min_periods = 1).apply(mean_notzero)
    df['area_mean/month'] = df['area'].rolling(30,min_periods = 1).apply(mean_notzero)
    df['area_mean/quarter'] = df['area'].rolling(90,min_periods = 1).apply(mean_notzero)
    df['area_mean/half-year'] = df['area'].rolling(180,min_periods = 1).apply(mean_notzero)

    df['area_high/2w'] = df['area'].rolling(14,min_periods = 1).max()
    df['area_high/month'] = df['area'].rolling(30,min_periods = 1).max()
    df['area_high/quarter'] = df['area'].rolling(90,min_periods = 1).max()
    df['area_high/half-year'] = df['area'].rolling(180,min_periods = 1).max()

    df['area_low/2w'] = df['area'].rolling(14,min_periods = 1).min()
    df['area_low/month'] = df['area'].rolling(30,min_periods = 1).min()
    df['area_low/quarter'] = df['area'].rolling(90,min_periods = 1).min()
    df['area_low/half-year'] = df['area'].rolling(180,min_periods = 1).min()
    
    df.to_csv(path + 'prep5/{}.csv'.format(str(i)))
    

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
D:\Anaconda3\lib\site-

In [7]:
i = 2
df = pd.DataFrame(columns = ['sku_id','order_time','sales'])
df['order_time'] = time_period
df['sku_id'] = i
df.fillna(0,inplace = True)

##sales
record = order[order['sku_id'] == i]
record = record[['o_date','o_sku_num']].groupby('o_date').sum()
date_list = list(record.index)
for date in date_list:
    df.loc[df['order_time'] == date,'sales'] = int(record.loc[date])
    
df['sales_sum/2w'] = df['sales'].rolling(14,min_periods = 1).sum()
df['sales_sum/month'] = df['sales'].rolling(30,min_periods = 1).sum()
df['sales_sum/quarter'] = df['sales'].rolling(90,min_periods = 1).sum()
df['sales_sum/half-year'] = df['sales'].rolling(180,min_periods = 1).sum()

df['sales_mean/2w'] = df['sales'].rolling(14,min_periods = 1).apply(mean_notzero)
df['sales_mean/month'] = df['sales'].rolling(30,min_periods = 1).apply(mean_notzero)
df['sales_mean/quarter'] = df['sales'].rolling(90,min_periods = 1).apply(mean_notzero)
df['sales_mean/half-year'] = df['sales'].rolling(180,min_periods = 1).apply(mean_notzero)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
D:\Anaconda3\lib\site-

In [11]:
###comment
comment['comment_time'] = pd.to_datetime(comment['comment_create_tm']).dt.date
order_id = order.loc[order['sku_id'] == i,:]['o_id']
order_id = list(order_id)
comment_record = comment[comment['o_id'].isin(order_id)]

comment_record = comment_record[comment_record['score_level'] != -1]
comment_record_mean = comment_record[['comment_time','score_level']].groupby('comment_time').mean()
comment_record_high = comment_record[['comment_time','score_level']].groupby('comment_time').min()
comment_record_low = comment_record[['comment_time','score_level']].groupby('comment_time').max()

df['comment_mean'] = 0
df['comment_high'] = 0
df['comment_low'] = 0

date_list = list(comment_record_mean.index)
for date in date_list:
    df.loc[df['order_time'] == date,'comment_mean'] = int(comment_record_mean.loc[date])
    df.loc[df['order_time'] == date,'comment_high'] = int(comment_record_high.loc[date])
    df.loc[df['order_time'] == date,'comment_low'] = int(comment_record_low.loc[date])
    
df['comment_mean/2w'] = df['comment_mean'].rolling(14,min_periods = 1).apply(mean_notzero)
df['comment_mean/month'] = df['comment_mean'].rolling(30,min_periods = 1).apply(mean_notzero)
df['comment_mean/quarter'] = df['comment_mean'].rolling(90,min_periods = 1).apply(mean_notzero)
df['comment_mean/half-year'] = df['comment_mean'].rolling(180,min_periods = 1).apply(mean_notzero)

df['comment_high/2w'] = df['comment_high'].rolling(14,min_periods = 1).min()
df['comment_high/month'] = df['comment_high'].rolling(30,min_periods = 1).min()
df['comment_high/quarter'] = df['comment_high'].rolling(90,min_periods = 1).min()
df['comment_high/half-year'] = df['comment_high'].rolling(180,min_periods = 1).min()

df['comment_low/2w'] = df['comment_low'].rolling(14,min_periods = 1).max()
df['comment_low/month'] = df['comment_low'].rolling(30,min_periods = 1).max()
df['comment_low/quarter'] = df['comment_low'].rolling(90,min_periods = 1).max()
df['comment_low/half-year'] = df['comment_low'].rolling(180,min_periods = 1).max()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not

In [13]:
##action
browse_record = action[(action['sku_id'] == i)&(action['a_type'] == 1)]
follow_record = action[(action['sku_id'] == i)&(action['a_type'] == 2)]

browse_record = browse_record[['a_date','a_num']].groupby('a_date').sum()
follow_record = follow_record[['a_date','a_num']].groupby('a_date').sum()

df['browse'] = 0
df['follow'] = 0
browse_date_list = list(browse_record.index)
for date in browse_date_list:
    df.loc[df['order_time'] == date,'browse'] = int(browse_record.loc[date])
df['browse/2w'] = df['browse'].rolling(14,min_periods = 1).sum()
df['browse/month'] = df['browse'].rolling(30,min_periods = 1).sum()
df['browse/quarter'] = df['browse'].rolling(90,min_periods = 1).sum()
df['browse/half-year'] = df['browse'].rolling(180,min_periods = 1).sum()

follow_date_list = list(follow_record.index)
for date in follow_date_list:
    df.loc[df['order_time'] == date,'follow'] = int(follow_record.loc[date])
df['follow/2w'] = df['follow'].rolling(14,min_periods = 1).sum()
df['follow/month'] = df['follow'].rolling(30,min_periods = 1).sum()
df['follow/quarter'] = df['follow'].rolling(90,min_periods = 1).sum()
df['follow/half-year'] = df['follow'].rolling(180,min_periods = 1).sum()

In [16]:
##user level
user_new = user[['user_id','user_lv_cd']]
user_order = pd.merge(order,user_new,on='user_id')
user_record = user_order[user_order['sku_id'] == i]
user_record = user_record[['o_date','user_lv_cd']].groupby('o_date').mean()

df['user'] = 0
user_date_list = list(user_record.index)
for date in user_date_list:
    df.loc[df['order_time'] == date,'user'] = int(user_record.loc[date])
df['user/2w'] = df['user'].rolling(14,min_periods = 1).apply(mean_notzero)
df['user/month'] = df['user'].rolling(30,min_periods = 1).apply(mean_notzero)
df['user/quarter'] = df['user'].rolling(90,min_periods = 1).apply(mean_notzero)
df['user/half-year'] = df['user'].rolling(180,min_periods = 1).apply(mean_notzero)

df['user_mean/2w'] = df['user'].rolling(14,min_periods = 1).mean()
df['user_mean/month'] = df['user'].rolling(30,min_periods = 1).mean()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  if sys.path[0] == '':
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current beha

In [18]:
##location
location = order[order['sku_id'] == i]
location_record = location.groupby('o_date').o_area.nunique()

df['area'] = 0
location_date_list = list(location_record.index)
for date in location_date_list:
    df.loc[df['order_time'] == date,'area'] = int(location_record[date])
df['area_mean/2w'] = df['area'].rolling(14,min_periods = 1).apply(mean_notzero)
df['area_mean/month'] = df['area'].rolling(30,min_periods = 1).apply(mean_notzero)
df['area_mean/quarter'] = df['area'].rolling(90,min_periods = 1).apply(mean_notzero)
df['area_mean/half-year'] = df['area'].rolling(180,min_periods = 1).apply(mean_notzero)

df['area_high/2w'] = df['area'].rolling(14,min_periods = 1).max()
df['area_high/month'] = df['area'].rolling(30,min_periods = 1).max()
df['area_high/quarter'] = df['area'].rolling(90,min_periods = 1).max()
df['area_high/half-year'] = df['area'].rolling(180,min_periods = 1).max()

df['area_low/2w'] = df['area'].rolling(14,min_periods = 1).min()
df['area_low/month'] = df['area'].rolling(30,min_periods = 1).min()
df['area_low/quarter'] = df['area'].rolling(90,min_periods = 1).min()
df['area_low/half-year'] = df['area'].rolling(180,min_periods = 1).min()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  if __name__ == '__main__':
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  # Remove the CWD from sys.path while we load stuff.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current beha

In [19]:
df

,sku_id,order_time,sales,sales_sum/2w,sales_sum/month,sales_sum/quarter,sales_sum/half-year,sales_mean/2w,sales_mean/month,sales_mean/quarter,...,area_mean/quarter,area_mean/half-year,area_high/2w,area_high/month,area_high/quarter,area_high/half-year,area_low/2w,area_low/month,area_low/quarter,area_low/half-year
0,2,2016-05-01,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,2016-05-02,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2016-05-03,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,2016-05-04,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2,2016-05-05,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2,2017-04-27,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
362,2,2017-04-28,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
363,2,2017-04-29,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
364,2,2017-04-30,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
